In [1]:
import os
import requests

import pandas as pd
from dotenv import load_dotenv


load_dotenv()

av_key = os.getenv("ALPHA_VANTAGE_KEY")

def read_intraday_data(tickers: list[str], interval: int):
    dfs = []

    for ticker in tickers:
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={ticker}&interval={interval}min&apikey={av_key}"
        r = requests.get(url)
        data = r.json()

        time_series = data[f"Time Series ({interval}min)"]
        df = pd.DataFrame(time_series)
        dfs.append(df)

    return dfs

In [2]:
dfs = read_intraday_data(tickers=["IBM"], interval=15)

In [3]:
df = dfs[0]

In [4]:
df.head()

,2025-06-20 19:45:00,2025-06-20 19:30:00,2025-06-20 19:15:00,2025-06-20 19:00:00,2025-06-20 18:45:00,2025-06-20 18:30:00,2025-06-20 18:15:00,2025-06-20 18:00:00,2025-06-20 17:45:00,2025-06-20 17:30:00,...,2025-06-18 13:15:00,2025-06-18 13:00:00,2025-06-18 12:45:00,2025-06-18 12:30:00,2025-06-18 12:15:00,2025-06-18 12:00:00,2025-06-18 11:45:00,2025-06-18 11:30:00,2025-06-18 11:15:00,2025-06-18 11:00:00
1. open,282.6900,282.6900,281.3900,280.9700,281.3800,280.9700,281.2600,281.3800,281.3900,281.2700,...,285.7950,285.7500,285.4650,285.2500,286.4050,285.2300,285.3700,285.4600,285.2180,285.9400
2. high,282.6900,282.6900,283.0000,281.4000,281.4000,281.3800,281.2600,281.3900,281.3900,281.3900,...,286.0100,286.2500,286.1999,285.7500,286.4600,286.5399,285.4900,285.7800,285.7700,286.0800
3. low,281.2000,281.0000,281.3900,280.9700,281.1000,280.9700,280.4000,281.2600,280.0500,281.0000,...,285.5900,285.6500,285.3765,285.1704,285.0800,285.1078,285.0000,285.2918,285.1700,285.0900
4. close,282.0000,281.2000,281.4300,281.3900,281.1100,281.3700,281.0000,281.3900,281.3800,281.2600,...,285.8000,285.7600,285.7850,285.3188,285.2000,286.3600,285.2300,285.4350,285.4120,285.1800
5. volume,104,94,810,2838646,182,2829504,23,3,8915,71,...,59282,100975,66851,61225,114427,86566,78366,63124,81847,111830


In [5]:
# Step 1: Transpose the DataFrame so timestamps become the index
df_rotated = df.T

# Step 2: Clean up column names by removing the number prefixes (e.g., "1. open" → "open")
df_rotated.columns = [col.split('. ', 1)[-1] for col in df_rotated.columns]

# Step 3 (optional): Convert index to datetime if you want time-based operations
df_rotated.index = pd.to_datetime(df_rotated.index)

# Step 4 (optional): Sort by time (ascending)
df_rotated = df_rotated.sort_index()

# Final result
df_rotated.head()

,open,high,low,close,volume
2025-06-18 11:00:00,285.9400,286.0800,285.0900,285.1800,111830
2025-06-18 11:15:00,285.2180,285.7700,285.1700,285.4120,81847
2025-06-18 11:30:00,285.4600,285.7800,285.2918,285.4350,63124
2025-06-18 11:45:00,285.3700,285.4900,285.0000,285.2300,78366
2025-06-18 12:00:00,285.2300,286.5399,285.1078,286.3600,86566


In [6]:
df_rotated.tail()

,open,high,low,close,volume
2025-06-20 18:45:00,281.3800,281.4000,281.1000,281.1100,182
2025-06-20 19:00:00,280.9700,281.4000,280.9700,281.3900,2838646
2025-06-20 19:15:00,281.3900,283.0000,281.3900,281.4300,810
2025-06-20 19:30:00,282.6900,282.6900,281.0000,281.2000,94
2025-06-20 19:45:00,282.6900,282.6900,281.2000,282.0000,104


In [16]:
import plotly.graph_objects as go

# Assuming df_rotated is your transformed DataFrame
# with datetime index and columns: open, high, low, close

fig = go.Figure(data=[go.Candlestick(
    x=pd.to_datetime(df_rotated.index),
    open=df_rotated['open'].astype(float),
    high=df_rotated['high'].astype(float),
    low=df_rotated['low'].astype(float),
    close=df_rotated['close'].astype(float),
)])

# Customize layout
fig.update_layout(
    title='Candlestick Chart',
    xaxis_title='Time',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False,
    xaxis=dict(
        type='category',
    )
)

fig.show()

In [8]:
fig.write_image("candlestick.pdf")

In [9]:
fig.write_image("chart.svg")

In [ ]:
fig.write_image("chart.png", width=1920, height=1080, scale=2)

In [27]:
df_rotated.index[0].strftime('%Y-%m-%d %H:%M')

'2025-06-18 11:00'

In [26]:
df.index[0]

'1. open'